In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series, Timestamp
import tensorflow as tf
import tensorflow.compat.v1 as tf1
from tensorflow import Graph
from tensorflow.compat.v1 import Session, ConfigProto
from tensorflow.python.eager.context import PhysicalDevice
from typing import Dict, List, Union, Generator, Tuple, TypeVar, Type
import os
from numpy import load, ndarray
import time
from tensorflow.python.keras.engine.functional import Functional

In [2]:
_T = TypeVar("_T")

In [3]:
import sys
sys.path.insert(0, '..')

In [4]:
from data_formatters.base import GenericDataFormatter, InputTypes, DataTypes
from data_formatters.sorgenia_wind import SorgeniaFormatter

In [5]:
from expt_settings.configs import ExperimentConfig
from libs.hyperparam_opt import HyperparamOptManager
from libs.tft_model import TemporalFusionTransformer
import libs.utils as utils

In [6]:
def format_outputs(prediction: Union[List, ndarray]) -> DataFrame:
    """Returns formatted dataframes for prediction."""

    flat_prediction = pd.DataFrame(
        prediction[:, :, 0],
        columns=[
            't+{}'.format(i)
            for i in range(self.time_steps - self.num_encoder_steps)
        ])
    cols = list(flat_prediction.columns)
    flat_prediction['forecast_time'] = time[:, self.num_encoder_steps - 1, 0]
    flat_prediction['identifier'] = identifier[:, 0, 0]

    # Arrange in order
    return flat_prediction[['forecast_time', 'identifier'] + cols]

In [7]:
quantiles = [0.1, 0.5, 0.9]

In [8]:
class MyPredictor(object):
    def __init__(self, formatter: SorgeniaFormatter, params: Dict, config: ExperimentConfig):
        self.formatter = formatter
        self.params = params
        self.config = config
        print('Done.')
        
    def predict(self, instances: DataFrame, **kwargs) -> List[Tuple]:
        t0: float = time.perf_counter()
        if tf.test.gpu_device_name(): 
            print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
        else:
            print("Please install GPU version of TF")
        gpu: List[PhysicalDevice] = tf.config.experimental.list_physical_devices('GPU')
        tf.config.experimental.set_memory_growth(gpu[0], True)
        tf_config: ConfigProto = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)
        tf1.reset_default_graph()
        with tf.Graph().as_default(), tf1.Session(config=tf_config) as sess:
            tf1.keras.backend.set_session(sess)
            self.model = TemporalFusionTransformer(self.params)
            self.params.pop('exp_name', None)
            self.params.pop('data_folder', None)
            self.model.load(os.path.join(self.config.model_folder, "fixed"), use_keras_loadings=False)
            output_map: Dict = self.model.predict(instances, return_targets=False)
            # Extract predictions for each quantile into different entries
            preds: DataFrame = self.formatter.format_predictions(output_map.get("p50"))
        # convert output to a list if that's required by GCP
        t1: float = time.perf_counter()
        print("Time elapsed ", t1-t0)
        
        return [tuple(x) for x in preds.to_numpy()]
        
        
    @classmethod
    def from_path(cls: Type[_T], model_dir: str) -> _T:
        """
         :params : folder with model checkpoints and params
        """
        config = ExperimentConfig('sorgenia_wind', model_dir)
        formatter = config.make_data_formatter()
        print("Formatter data folder is ", formatter.data_folder)
        # Sets up default params
        fixed_params: Dict = formatter.get_experiment_params()
        params: Dict = formatter.get_default_model_params()
        params["model_folder"]: str = os.path.join(config.model_folder, "fixed")
        model_folder = os.path.join(config.model_folder, "fixed")
        # Sets up hyperparam manager
        print("*** Loading hyperparm manager ***")
        opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                           fixed_params, model_folder)
        params: Dict = opt_manager.get_next_parameters()
        params['exp_name'] = 'sorgenia_wind'
        params['data_folder'] = config.data_csv_path
        
        # load scalers
        formatter.load_scalers()
        
        return cls(formatter, params, config)

In [9]:
predictor = MyPredictor.from_path(r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs')

Formatter data folder is  C:\Users\Lorenzo\PycharmProjects\TFT\outputs\saved_models\sorgenia_wind
*** Loading hyperparm manager ***
Done.


In [10]:
# load inference sample
sample: DataFrame = pd.read_csv(r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs\data\sorgenia_wind\data\sorgenia_wind\data\sorgenia_wind_inference_sample.csv')
sample

,plant_name_up,time,kwh,dew_point_2m_C,temperature_2m_C,msl_pressure_hPa,sfc_pressure_hPa,precipitation_1h_mm,wind_speed_mean_10m_1h_ms,wind_speed_mean_100m_1h_ms,...,days_from_start,id,hour,day,day_of_week,month,categorical_id,hours_from_start,categorical_day_of_week,categorical_hour
0,UP_MPNTLCDMRN_1,2020-08-06 23:00:00,-0.529162,0.866993,0.239003,-0.048342,-1.274784,-0.191301,-0.733072,-0.050166,...,1314,UP_MPNTLCDMRN_1,1.661345,6,0.000959,8,0,2.726227,3,23
1,UP_MPNTLCDMRN_1,2020-08-07 00:00:00,-0.576865,0.866993,0.197319,-0.101425,-1.304034,-0.191301,-0.733072,-0.127810,...,1314,UP_MPNTLCDMRN_1,-1.661381,7,0.500952,8,0,2.726368,4,0
2,UP_MPNTLCDMRN_1,2020-08-07 01:00:00,-0.642457,0.904884,0.127844,-0.119119,-1.304034,-0.191301,-0.792745,-0.321922,...,1314,UP_MPNTLCDMRN_1,-1.516914,7,0.500952,8,0,2.726510,4,1
3,UP_MPNTLCDMRN_1,2020-08-07 02:00:00,-0.666309,0.866993,0.072265,-0.136814,-1.313784,-0.191301,-0.792745,-0.399567,...,1314,UP_MPNTLCDMRN_1,-1.372448,7,0.500952,8,0,2.726651,4,2
4,UP_MPNTLCDMRN_1,2020-08-07 03:00:00,-0.691651,0.848048,0.044475,-0.136814,-1.323534,-0.191301,-0.792745,-0.360744,...,1314,UP_MPNTLCDMRN_1,-1.227982,7,0.500952,8,0,2.726793,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,UP_PRCLCMINEO_1,2020-08-14 06:00:00,-0.689231,0.003895,1.212904,-0.084582,-0.357503,-0.159763,0.042203,0.070870,...,1321,UP_PRCLCMINEO_1,-0.794583,14,0.500952,8,6,2.750967,4,6
1256,UP_PRCLCMINEO_1,2020-08-14 07:00:00,-0.694629,-0.413023,1.674744,-0.120673,-0.393239,-0.159763,0.042203,-0.183379,...,1321,UP_PRCLCMINEO_1,-0.650116,14,0.500952,8,6,2.751108,4,7
1257,UP_PRCLCMINEO_1,2020-08-14 08:00:00,-0.684232,-0.602531,1.949352,-0.084582,-0.339634,-0.159763,0.106010,-0.310504,...,1321,UP_PRCLCMINEO_1,-0.505650,14,0.500952,8,6,2.751249,4,8
1258,UP_PRCLCMINEO_1,2020-08-14 09:00:00,0.103824,-0.356170,2.149067,-0.084582,-0.321766,-0.159763,0.488851,0.028495,...,1321,UP_PRCLCMINEO_1,-0.361184,14,0.500952,8,6,2.751391,4,9


In [11]:
df_maserio = sample[sample['id']=='UP_MPNTLCDMRN_1']
df_maserio['kwh'].shape

(180,)

In [12]:
df_maserio['kwh'].tail()

175   -0.737864
176   -0.561958
177   -0.078814
178    0.070259
179   -0.275739
Name: kwh, dtype: float64

In [ ]:
# testing Predictor on sample
preds: List = predictor.predict(sample)

In [ ]:
preds